# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint 
# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = pd.read_csv("../Weather/cities.csv")
data

,Unnamed: 0,City,lat,lon,temp,temp_max,humidity,cloudiness,wind_speed,country
0,0,Mananara,-16.17,49.77,79.72,79.72,83.0,98.0,5.84,MG
1,1,Castro,-24.79,-50.01,66.36,66.36,54.0,81.0,5.86,BR
2,2,Albany,42.60,-73.97,36.90,39.99,69.0,75.0,0.81,US
3,3,Huarmey,-10.07,-78.15,73.45,73.45,77.0,99.0,4.07,PE
4,4,Chokurdakh,70.63,147.92,-0.44,-0.44,92.0,66.0,2.75,RU
...,...,...,...,...,...,...,...,...,...,...
541,606,Macia,-25.03,33.10,76.71,76.71,60.0,17.0,9.64,MZ
542,607,Brae,60.40,-1.35,37.99,39.20,55.0,60.0,24.16,GB
543,609,Zlobin,45.29,14.65,62.46,66.20,29.0,39.0,8.05,HR
544,610,Severodvinsk,64.56,39.83,42.80,42.80,56.0,40.0,8.95,RU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = data[["lat", "lon"]].astype(float)
humidity = data[["humidity"]]

humidity = pd.to_numeric(humidity["humidity"], downcast = "float")

In [4]:
fig = gmaps.figure()

heat_map = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(heat_map)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_data = data.loc[(data["temp_max"]>70) & (data["temp_max"] < 80) & (data["wind_speed"] < 10) & (data["cloudiness"] == 0)]
ideal_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 21 to 384
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  6 non-null      int64  
 1   City        6 non-null      object 
 2   lat         6 non-null      float64
 3   lon         6 non-null      float64
 4   temp        6 non-null      float64
 5   temp_max    6 non-null      float64
 6   humidity    6 non-null      float64
 7   cloudiness  6 non-null      float64
 8   wind_speed  6 non-null      float64
 9   country     6 non-null      object 
dtypes: float64(7), int64(1), object(2)
memory usage: 528.0+ bytes


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_data

hotel_df["Hotel_Name"] = ""
hotel_df



params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():

    lat = row["lat"]
    lon = row["lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel = requests.get(base_url, params=params)
    

    hotels = hotel.json()

    try:
        hotel_df.loc[index, "Hotel_Name"] = hotels["results"][0]["name"]
        print(hotels["results"][0]["name"])
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Hotel Polo Orchid
Hosteria de la Cascada
Abacus Motel
Hotel Kununurra
The Halyards Hotel
Missing field/result... skipping.


,Unnamed: 0,City,lat,lon,temp,temp_max,humidity,cloudiness,wind_speed,country,Hotel_Name
21,26,Tura,25.52,90.22,76.95,76.95,34.0,0.0,4.05,IN,Hotel Polo Orchid
96,109,Tandil,-37.32,-59.13,75.00,75.00,72.0,0.0,1.01,AR,Hosteria de la Cascada
111,127,Mount Isa,-20.73,139.50,77.00,77.00,50.0,0.0,9.08,AU,Abacus Motel
133,149,Kununurra,-15.77,128.73,77.00,77.00,69.0,0.0,2.24,AU,Hotel Kununurra
150,168,Port Alfred,-33.59,26.89,73.00,73.00,78.0,0.0,1.01,ZA,The Halyards Hotel
384,436,Marakkanam,12.20,79.95,79.72,79.72,71.0,0.0,9.82,IN,


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

locations = hotel_df[["lat", "lon"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))